# 1st Iteration

At first, let us try the simplest approach - our baseline - do not handle emojis at all (Treat them as UNK). The main purpose of this iteration is to check whether emoji add any emotional value to the text. 

Main steps:
1. Choose emoji-rich dataset
2. Delete emojis from it 
3. Create embeddings from input text
4. Search for the most relevant text from database
5. Drive conclusions about actual relevance using metrics